In [3]:
# ================================
# Setup: Install and Import Libraries
# ================================
!pip install -q peft

import os
import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
from torch.optim import AdamW
# Evaluation metrics
!pip install -q sacrebleu bert-score evaluate unbabel-comet

import sacrebleu
from bert_score import score as bert_score
import evaluate
from comet import download_model, load_from_checkpoint


# PEFT (LoRA)
from peft import LoraConfig, get_peft_model, TaskType


2025-12-19 10:14:29.633390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766139269.810610      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766139269.862036      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766139270.291987      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766139270.292022      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766139270.292024      55 computation_placer.cc:177] computation placer alr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 118.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 34.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [4]:
# Enter Access Token and rerun
from huggingface_hub import login
login(new_session=False)

In [6]:
# ================================
# Load Base Model and Tokenizer
# ================================
ckpt = "ai4bharat/indictrans2-indic-en-1B"
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(ckpt)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("Model loaded on", device)


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

dict.TGT.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

model.SRC:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

model.TGT:   0%|          | 0.00/759k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Model loaded on cuda


In [21]:
# ================================
# Prepare Domain-Specific Data
# ================================

raw_data = load_dataset("atrisaxena/mini-iitb-english-hindi")
raw_data = raw_data["train"]  # use train split

# 500 statements --> 10,0000,1000 was giving error -- i tried but error was there --i tried by eliminating empty example then also it showed invalid source language tag
domain_train = raw_data.shuffle(seed=42).select(range(500))
domain_val   = raw_data.shuffle(seed=42).select(range(50, 75))  # small dev set



In [22]:
# ================================
# Tokenization Function (IndicTrans2 – ACTUAL FINAL FIX)
# ================================


SRC_TAG = "hin_Deva"
TGT_TAG = "eng_Latn"

source_lang = "hi"
target_lang = "en"

max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = []
    targets = []

    for ex in examples["translation"]:
        # extract safely
        src = ex.get(source_lang)
        tgt = ex.get(target_lang)

        if src is None or tgt is None:
            continue

        src = src.strip()
        tgt = tgt.strip()

        if not src or not tgt:
            continue

        # STRICT IndicTrans2 format: exactly two tags
        inputs.append(f"{SRC_TAG} {TGT_TAG} {src}")
        targets.append(tgt)

    # 🚨 ABSOLUTELY REQUIRED
    # HuggingFace tokenizer + IndicTrans2 CANNOT handle empty input lists
    if len(inputs) == 0:
        return {}

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    # Labels: NO language tags
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [23]:
tokenized_train = domain_train.map(
    preprocess_function,
    batched=True,
    remove_columns=domain_train.column_names
)

In [24]:
tokenized_val = domain_val.map(
    preprocess_function,
    batched=True,
    remove_columns=domain_val.column_names
)

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [25]:
print(domain_train[0])


{'translation': {'en': 'memento', 'hi': 'अभिज्ञा'}}


In [26]:
# DataLoader setup
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
train_loader = DataLoader(tokenized_train, batch_size=16, shuffle=True, collate_fn=data_collator, num_workers=2)
val_loader   = DataLoader(tokenized_val,   batch_size=16, shuffle=False, collate_fn=data_collator, num_workers=1)


In [27]:
# ================================
# Configure LoRA Adapters
# ================================
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=4,                     
    lora_alpha=8,            # scaling
    lora_dropout=0.05,
    bias="none",

    # ONLY query & value projections
    target_modules=[
        "q_proj",
        "v_proj"
    ]
)
model.gradient_checkpointing_enable()
model.config.use_cache = False


# Wrap the model with LoRA adapters
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # (Optional) shows how many params are trainable


You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 884,736 || all params: 1,023,891,456 || trainable%: 0.0864


In [28]:
# ================================
# Training Loop with LoRA Fine-Tuning (AMP-safe)
# ================================

from torch.amp import autocast, GradScaler
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scaler = GradScaler("cuda")

num_epochs = 1
model.train()

for epoch in range(num_epochs):
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        # Move batch to GPU
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad(set_to_none=True)

        # Mixed Precision Forward Pass
        with autocast("cuda"):
            outputs = model(**batch)
            loss = outputs.loss

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f}")

    # ================================
    # Validation
    # ================================
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with autocast("cuda"):
                outputs = model(**batch)
                val_loss += outputs.loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1} | Validation Loss: {val_loss:.4f}")

    model.train()

print(" LoRA Fine-tuning complete.")


Training Epoch 1:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch 1 | Train Loss: 11.0724
Epoch 1 | Validation Loss: 10.9076
 LoRA Fine-tuning complete.


In [29]:
from huggingface_hub import login

login()  # paste your HF token

In [30]:
repo_id = "Vir123-dev/indictrans2_hi_en_finetune_1B"


In [31]:
# Save LoRA adapter + push to hub
model.push_to_hub(
    repo_id,
    commit_message="LoRA fine-tuned IndicTrans2 on Domain-1 (HI-EN)"
)

# Save tokenizer (IMPORTANT)
tokenizer.push_to_hub(repo_id)


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/Vir123-dev/indictrans2_hi_en_finetune_1B/commit/001d6db1c7c69461e0dc07f4f77727c6e6aafeb2', commit_message='Upload tokenizer', commit_description='', oid='001d6db1c7c69461e0dc07f4f77727c6e6aafeb2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Vir123-dev/indictrans2_hi_en_finetune_1B', endpoint='https://huggingface.co', repo_type='model', repo_id='Vir123-dev/indictrans2_hi_en_finetune_1B'), pr_revision=None, pr_num=None)